# Analysis of Variations
Running through the Penn State <a href="https://onlinecourses.science.psu.edu/stat502">Stats 502</a> course online.

In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import pandas as pd
import scipy as sc

In [2]:
data = pd.read_csv("data/sampleData.csv", header=0)
print(data.to_string(index=False))  
data.describe()

Control    F1    F2    F3
   21.0  32.0  22.5  28.0
   19.5  30.5  26.0  27.5
   22.5  25.0  28.0  31.0
   21.5  27.5  27.0  29.5
   20.5  28.0  26.5  30.0
   21.0  28.6  25.2  29.2


,Control,F1,F2,F3
count,6.000,6.000000,6.000000,6.00000
mean,21.000,28.600000,25.866667,29.20000
std,1.000,2.437212,1.899123,1.28841
min,19.500,25.000000,22.500000,27.50000
25%,20.625,27.625000,25.400000,28.30000
50%,21.000,28.300000,26.250000,29.35000
75%,21.375,30.025000,26.875000,29.87500
max,22.500,32.000000,28.000000,31.00000


In [3]:
summary = data.describe()
summary.loc[['mean', '50%']]

,Control,F1,F2,F3
mean,21.0,28.6,25.866667,29.20
50%,21.0,28.3,26.250000,29.35



#### Start our analysis by considering the total variance of the response variable ${Y_{ij}}$

In [93]:
#np.var actually returns the mean squared error
total_mean = np.mean(data.values)
total_ss = np.sum((data.values - total_mean)**2)
print("total sum of square errors = {:.2f}".format(total_ss))

total sum of square errors = 312.47


#### Sum of square deviations for each treatment (each column in data).  This is the difference between the treatment means and the total mean.  We expect this to be relatively large if our treatment had any effects.

In [94]:
#get the mean for each column (treatment)
treatment_stats = pd.DataFrame({'means':data.mean().values, 'count':data.count()})

#caculate the difference between the treament mean and the total mean
treatment_residuals = (treatment_stats['means'].values - total_mean)

#the sum of the squares of the treatment deviations, weighted by the number of instances
#each treatment
treatment_ss = np.sum((treatment_residuals**2) * treatment_stats['count'].values)
print("treatment sum of square errors = {:.2f}".format(treatment_ss))


treatment sum of square errors = 251.44


####  Finally calculate the variability left over, the sum square of all residuals from the treatment means.  This is the difference between the treatment and the treatment mean.  We expect this to be relatively small

In [95]:
#will subtract treatment_stats['means'] as a row from each row of data.values
treatment_residuals = data.values - treatment_stats['means'].values
toString = lambda x: "{:.2f}".format(x)

print("treatment residuals")
for i in range(0,data.shape[1]):
    print(str(data.columns[i]) + ":  "+ str(map(toString, treatment_residuals[:,i])))
    
error_ss = np.sum(treatment_residuals**2)  
print("\nerror_ss = {:.3f}".format(error_ss))

treatment residuals
Control:  ['0.00', '-1.50', '1.50', '0.50', '-0.50', '0.00']
F1:  ['3.40', '1.90', '-3.60', '-1.10', '-0.60', '0.00']
F2:  ['-3.37', '0.13', '2.13', '1.13', '0.63', '-0.67']
F3:  ['-1.20', '-1.70', '1.80', '0.30', '0.80', '0.00']

error_ss = 61.033


Here we will use the definition of variance

$var ~=~ {\sum{(X_i - \overline{X})^2} \over {N-1}} ~=~ {{SS}\over{df}}$


In [96]:
#there are four means, but they must average to the total mean
treatment_df = 3

#there are 24 independent values, but each treatment must average to its respective average.  
#thus, 24-4 = 20
error_df = 20

#finally, the total ss, there are 24 values, but they must average to the total avergag, 24-1
total_df = 23

#mean square errors
treatment_MSE = treatment_ss / treatment_df
print("treatment_MSE = {:.3f}".format(treatment_MSE))

error_MSE = error_ss / error_df
print("error_MSE = {:.3f}".format(error_MSE))


total_MSE = total_ss / total_df
print("total_MSE = {:.3f}".format(total_MSE))



treatment_MSE = 83.813
error_MES = 3.052
total_MSE = 13.586


#### the F-statistic is the treatment MSE over the error MSE.  That is, the variability due to our treatment over the error (or variability within our treatment).  Intuitivively, we want this to be large, because if the variability within our treatments is of the order of the variability between treatments, then  there is a large chance that the variability seen between treatments is due to the general variability.

In [99]:
F = treatment_MSE / error_MSE
print("F-statistic = {:.2f}".format(F))

F-statistic = 27.46


Our critical value for rejecting the null hypothesis is: $F_{\alpha} = F_{(0.05, 3, 20)} = 3.10$.  That is, there is a 95% chance, that four groups, with 24 measurements will have a ratio of MSE_accross_groups/MSE_within_groups less than $F_{\alpha}$ if there is no difference between the groups.  
  
Our F-statistic (27.46) is obviously much larger than $F_{\alpha = 0.05} = 3.10$.  Our p-value is $2.6$x$10^{-7}$

### A Review of the F-test